In [1]:
import model.ChainCRF
import importlib
import dataLoad.lload
importlib.reload(model.ChainCRF)
importlib.reload(dataLoad.lload)
from  dataLoad.lload import CustomDataset
from  dataLoad.lload import *
import numpy as np
from model.ChainCRF import BLITM,ChainCRF
import torchtext
from get_data import *
from check import *
from collections import Counter,defaultdict
embed_size = 100
language = "English"
min_freq = 4 if language == "Chinese" else 2
##Chinese 300 100 10,1e-3  Loss:281.3659775416056 micro avg  0.9368 
##Chinese 100 100 10,1e-3  Loss:281.3659775416056 micro avg  0.9447 
##Chinese 50 100 10,1e-4  Loss:281.3659775416056 micro avg 0.6839 
##Chinese 50 100 10,5e-4  Loss:277.4861424763997 micro avg 0.9065
## 0.7683 
## English
##  pretrain 8317 +0.8276 =0.8320  1e-3 10
##  pretrain 8317 +0.8276 =0.8347  1e-3 10 zeros
##  pretrain 8317 +0.8276 =0.8399  1e-4 10 zeros
#   0.8381 


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BiLSTM_CRF(nn.Module):
    def __init__(self, num_classes, vocab_length,embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.bilstm = BLITM(num_classes,vocab_length,embedding_dim,hidden_dim)
        self.crf = ChainCRF(num_classes)

    def forward(self, sentence, mask, targets=None, pre_train=None):
    
        emissions = self.bilstm(sentence)
        if targets is not None:
            # 计算CRF损失
            # mask = (sentence != 0)  # 使用 0 填充的词的位置作为掩码
            crf_loss = self.crf(emissions, targets, mask)
            return crf_loss
        else:
            # 测试时，使用维特比解码
            tags = self.crf.viterbi_decode(emissions, mask)
            return tags


In [3]:

train_data = get_train_data(language)
import os
import torchtext
train_word = [word  for sentence in train_data for word,label in sentence]
print(len(train_word))
vocab = torchtext.vocab.vocab(Counter(train_word),min_freq=min_freq,specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])
print(vocab.get_default_index())
def sent2word(sentence):
    return [w for w, _ in sentence]
def sent2label(sentence):
    return [l for _, l in sentence]
max_length = max([len(l) for l in train_data])
max_length = max(max_length, 256)
sorted_labels = sorted_labels_chn if language == 'Chinese' else sorted_labels_eng
def label2index(label):
    return sorted_labels.index(label)

123967
0


In [4]:

# Create a custom dataset
custom_dataset = CustomDataset(train_data, vocab, label2index, max_length)
# Create a DataLoader
batch_size = 32
dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

custom_dataset[0]

{'word_embeddings': tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
         19,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         

In [5]:
import torch
import torch.optim as optim
from tqdm import tqdm

def train(model, train_loader, num_epochs, learning_rate, device):
    # Move the model to GPU
    model.to(device)

    # Define the optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Create a progress bar
    progress_bar = tqdm(total=num_epochs * len(train_loader))

    for epoch in range(num_epochs):
        model.train()
        sum_loss = 0

        for batch in train_loader:
            # Move inputs, labels, and mask to GPU
            length = batch['max_length']
            max_length = np.argmax(length)
            aaa = length[max_length]
            inputs = batch['word_embeddings'][:,:aaa].to(device)
            labels = batch['label_indices'][:,:aaa].to(device)
            mask =  batch['mask'].to(device)[:,:aaa].to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            loss = model(inputs, mask, labels)

            # Backward pass
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1)
            sum_loss += loss.item()

            # Update the parameters
            optimizer.step()
            progress_bar.update(1)

        progress_bar.set_postfix_str("Epoch:{}, Loss:{}".format(epoch + 1, sum_loss / len(train_loader)))
        print(" ")

    # Close the progress bar
    progress_bar.close()

# Example usage:
# Assuming you have a model, train_loader, num_epochs, and learning_rate



In [6]:
hidden_dim = 100
bilstm_crf = BiLSTM_CRF(len(sorted_labels),len(vocab),embed_size,hidden_dim)
pretrain_file = "./bilstm_crf/pretrain/BILSTM_{}.bin".format(language)
file = "BILSTM_CRF_{}.bin".format(language)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# bilstm_crf.bilstm.load_state_dict(torch.load(pretrain_file))
mode = True
if mode:
    train(bilstm_crf,dataloader,10,1e-3,device)
    torch.save(bilstm_crf.state_dict(),file)
else:
    bilstm_crf.load_state_dict(torch.load(file))

 10%|█         | 121/1200 [00:18<01:56,  9.29it/s, Epoch:1, Loss:329.22883129119873]

 20%|██        | 241/1200 [00:36<02:05,  7.63it/s, Epoch:2, Loss:296.39825750986734]

 30%|███       | 361/1200 [00:54<01:46,  7.84it/s, Epoch:3, Loss:279.52405109405515]

 40%|████      | 481/1200 [01:11<01:40,  7.13it/s, Epoch:4, Loss:280.946046257019]  

 50%|█████     | 601/1200 [01:30<01:05,  9.15it/s, Epoch:5, Loss:281.3986678441366]

 60%|██████    | 720/1200 [01:48<01:20,  5.94it/s, Epoch:6, Loss:280.7885049819946]

 70%|███████   | 841/1200 [02:06<00:42,  8.43it/s, Epoch:7, Loss:282.91113929748536]

 80%|████████  | 961/1200 [02:25<00:25,  9.55it/s, Epoch:8, Loss:279.2567253748576] 

 90%|█████████ | 1081/1200 [02:43<00:16,  7.33it/s, Epoch:9, Loss:283.78191502888996]

100%|██████████| 1200/1200 [03:01<00:00,  6.62it/s, Epoch:10, Loss:281.18672478993733]

In [7]:
def mycheck(language, vocab, res_file, model, train_or_valid,device):
    valid = get_data_from_file(res_file)
    pred_path = "example_data/BILSTM_CRF_{}_{}.txt".format(language, "train" if train_or_valid else "dev")
    valid_data = CustomDataset(valid, vocab, label2index, 256)
    valdataloader = DataLoader(valid_data, batch_size=64, shuffle=False)

    # Move the model to GPU
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    with open(pred_path, "w") as f:
        with torch.no_grad():
            iter = 0
            for batch in valdataloader:
                # Move inputs, labels, and mask to GPU
                length = batch['max_length']
                max_length = np.argmax(length)
                aaa = length[max_length]
                word_embeddings = batch['word_embeddings'][:,:aaa]
                masks= batch['mask'][:,:aaa]

                preds = model(word_embeddings, masks)

                for pred in preds:
                    pred_labels = []
                    for i in range(len(valid[iter])):
                        f.write(valid[iter][i][0] + " " + sorted_labels[pred[i]] + '\n')
                        pred_labels.append(sorted_labels[pred[i]])
                    f.write('\n')
                    iter = iter + 1

    # Move the model back to CPU if needed
    model.to("cpu")

    check(language, "{}/{}.txt".format(language, "train" if train_or_valid else "validation"), pred_path)

# Example usage:
# Assuming you have a model, vocab, res_file, max_length, train_or_valid, label2index, sorted_labels
# Make sure to replace the imports with the actual modules and functions in your code


In [8]:
device = torch.device("cpu")

mycheck(language=language,vocab=vocab,res_file="{}/train.txt".format(language),model=bilstm_crf,train_or_valid=1,device=device)
mycheck(language=language,vocab=vocab,res_file="{}/validation.txt".format(language),model=bilstm_crf,train_or_valid=0,device=device)
# loss:114 0.3803  0.3954
# -92.54755198711143 0.3460 0.3643

              precision    recall  f1-score   support

      B-NAME     0.9942    0.9907    0.9924       861
      M-NAME     0.9946    0.9905    0.9926       740
      E-NAME     0.9942    0.9907    0.9924       861
      S-NAME     0.9674    0.9780    0.9727        91
      B-CONT     0.9961    0.9923    0.9942       260
      M-CONT     0.9959    0.9800    0.9879       499
      E-CONT     0.9846    0.9808    0.9827       260
      S-CONT     0.0000    0.0000    0.0000         0
       B-EDU     0.9919    0.9930    0.9924       858
       M-EDU     0.9902    0.9915    0.9909      1536
       E-EDU     0.9929    0.9837    0.9883       858
       S-EDU     0.0000    0.0000    0.0000         0
     B-TITLE     0.9866    0.9840    0.9853      6296
     M-TITLE     0.9857    0.9881    0.9869     14813
     E-TITLE     0.9960    0.9933    0.9947      6296
     S-TITLE     0.0000    0.0000    0.0000         0
       B-ORG     0.9934    0.9872    0.9903      4603
       M-ORG     0.9946    